In [ ]:
from collections import OrderedDict
from chempy import Substance, Equilibrium, Reaction, ReactionSystem
from chempy.kinetics.rates import MassAction
from chempy.thermodynamics.expressions import EqExpr
from chempy.util.graph import rsys2graph

In [ ]:
substances = OrderedDict([
    ('N', Substance('N', composition={'protein': 1})),
    ('U', Substance('U', composition={'protein': 1})),
    ('A', Substance('A', composition={'protein': 1})),
    ('L', Substance('L', composition={'ligand': 1})),
    ('NL', Substance('NL', composition={'protein': 1, 'ligand': 1})),
])

In [ ]:
R = 8.314472
kB_h = 1.3806504e-23 / 6.62606896e-34  # 2.083664399411865e10 K**-1 * s**-1

def _gibbs(args, T, backend):
    H, S, Cp, Tref = args
    H2 = H + Cp*(T - Tref)
    S2 = S + Cp*backend.log(T/Tref)
    return backend.exp(-(H2 - T*S2)/(R*T))

def _eyring(args, T, backend):
    H, S = args
    return kb_h/T*backend.exp(-(H - T*S)/(R*T))

In [ ]:
Gibbs = EqExpr.from_callback(_gibbs, parameter_keys=('temperature',), argument_names=('H', 'S', 'Cp', 'Tref'))
Eyring = MassAction.from_callback(_eyring, parameter_keys=('temperature',), argument_names=('H', 'S'))

In [ ]:
thermo_dis = Gibbs(unique_keys=('He_dis', 'Se_dis', 'Cp_dis', 'Tref_dis'))
thermo_u = Gibbs(unique_keys=('He_u', 'Se_u', 'Cp_u', 'Tref_u'))  # ([He_u_R, Se_u_R, Cp_u_R, Tref])
kinetics_agg = Eyring(unique_keys=('Ha_agg', 'Sa_agg'))  # EyringMassAction([Ha_agg, Sa_agg])
kinetics_as = Eyring(unique_keys=('Ha_as', 'Sa_as'))
kinetics_f = Eyring(unique_keys=('Ha_f', 'Sa_f'))

In [ ]:
eq_dis = Equilibrium({'NL'}, {'N', 'L'}, thermo_dis, name='ligand-protein dissociation')
eq_u = Equilibrium({'N'}, {'U'}, thermo_u, {'L'}, {'L'}, name='protein unfolding')
r_agg = Reaction({'U'}, {'A'}, kinetics_agg, {'L'}, {'L'}, name='protein aggregation')

In [ ]:
rsys = ReactionSystem(
    eq_dis.as_reactions(kb=kinetics_as, new_name='ligand-protein association') +
    eq_u.as_reactions(kb=kinetics_f, new_name='protein folding') +
    (r_agg,), substances, name='4-state CETSA system')
rsys

In [ ]:
vecs, comp = rsys.composition_balance_vectors()
names = rsys.substance_names()
dict(zip(comp, [dict(zip(names, v)) for v in vecs]))

In [ ]:
rsys2graph(rsys, '4state.png', save='.', include_inactive=False)
from IPython.display import Image; Image('4state.png')

In [ ]:
from IPython.display import HTML
HTML('1st order processes\n' + rsys.unimolecular_html_table()[0] +
     '<br><br>2nd order processes\n' + rsys.bimolecular_html_table()[0])